In [ ]:
# 全局开关：True 表示启用自动追加到文件，False 表示关闭
write_to_file = False  # 初始默认关闭，需要时手动改为 True

# 目标文件路径（固定为 Assign1-1.py）
target_file = "Assign1-1.py"

def auto_write(cell_content):
    """
    自动执行代码并根据开关决定是否追加到文件
    参数：cell_content (str)：当前代码块的内容（用三引号包裹）
    """
    # 1. 先执行当前代码（确保功能正常）
    exec(cell_content, globals())
    
    # 2. 如果开关开启，追加到目标文件
    if write_to_file:
        from IPython import get_ipython
        get_ipython().run_cell_magic('file', f'-a {target_file}', cell_content)
        print(f"已追加到 {target_file}")
    else:
        print("未追加到文件（开关关闭）")

# 安装依赖包和引用包

In [29]:
auto_write("""
#!/usr/bin/env python3

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

from nltk.stem import PorterStemmer

from nltk.corpus import wordnet, stopwords

from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()

from nltk.probability import FreqDist
""")

ℹ️ 未追加到文件（开关关闭）


[nltk_data] Downloading package punkt to /Users/hqyang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/hqyang/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hqyang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/hqyang/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [30]:
auto_write(""" 
import requests
from urllib.parse import urlparse

import re
from bs4 import BeautifulSoup

import contractions

import unicodedata

from opencc import OpenCC
import jieba
cc_zh = OpenCC('t2s')  # 中文繁体转简体
""")

ℹ️ 未追加到文件（开关关闭）


In [31]:
auto_write("""
from tqdm import tqdm  # 导入tqdm库
import argparse
import json
from pathlib import Path  # 用于路径验证
""")

ℹ️ 未追加到文件（开关关闭）


# 检测语言类型 (仅支持中文、英文)

In [32]:
auto_write("""
def detect_language(text):
    '''
    更严谨的实现是使用langdetect包，但该包有时会误判中英混合文本为其他语言
    需要安装：pip install langdetect
    from langdetect import detect, LangDetectException

    def detect_language(text):
        if not text.strip():
            return 'unknown'
        try:
            lang = detect(text)
            # 映射为中文（zh）或英文（en），其他语言返回unknown
            return 'zh' if lang == 'zh-cn' else 'en' if lang == 'en' else 'unknown'
        except LangDetectException:
            return 'unknown'
    '''
    if not text.strip():
        return 'unknown'
    
    # 提取中文CJK字符和英文单词字符
    cjk_chars = re.findall(r'[\\u4e00-\\u9fff]', text)
    en_chars = re.findall(r'[a-zA-Z]', text)  # 英文特征：字母
    
    total = len(text.strip())
    if total == 0:
        return 'unknown'
    
    cjk_ratio = len(cjk_chars) / total
    en_ratio = len(en_chars) / total
    
    # 中文判定：CJK占比超30%
    if cjk_ratio > 0.3:
        return 'zh'
    # 英文判定：英文占比超30%，且CJK占比极低（避免中英混合文本误判）
    elif en_ratio > 0.3 and cjk_ratio < 0.1:
        return 'en'
    # 其他情况（如中英混合、其他语言）
    else:
        return 'unknown'
""")

ℹ️ 未追加到文件（开关关闭）


In [33]:
auto_write("""
def get_statistics(text, n=10, k=10, lang='en'):
    '''
    获取文本中前n个高频词和长度最长的k个词
    
    参数：
        text: 输入文本
        n: 高频词数量（默认10）
        k: 最长词数量（默认10）
        lang: 语言（'en'英文/'zh'中文，默认'en'）
    
    返回：
        元组 (top_n_words, longest_k_words)，其中：
            - top_n_words: 列表，元素为 (词, 频率) 元组，按频率降序排列
            - longest_k_words: 列表，元素为 (词, 长度) 元组，按长度降序排列（长度相同则按词本身排序）
    '''    
    if not text.strip():
        return ([], [])  # 空文本返回两个空列表
    
    # 1. 分词（中英文适配）
    if lang == 'en':
        tokens = nltk.word_tokenize(text)
    elif lang == 'zh':
        tokens = jieba.lcut(text)  # 中文精确分词
    else:
        raise ValueError("Unsupported language. Use 'en' or 'zh'.")
    
    # 过滤空字符串（移除纯空格/空的token）
    tokens = [token.strip() for token in tokens]
    tokens = [token for token in tokens if token]  # 保留非空token
    
    # 2. 计算前n个高频词（复用原逻辑）
    fdist = FreqDist(tokens)
    top_n_words = fdist.most_common(n)  # 格式：[(词1, 频率1), (词2, 频率2), ...]
    
    # 3. 计算长度最长的k个词（去重后按长度排序）
    unique_tokens = list(set(tokens))  # 去重，避免重复词占据位置
    # 按长度降序排序（长度相同则按词的字母/字符顺序排序，保证稳定性）
    sorted_by_length = sorted(
        unique_tokens,
        key=lambda x: (-len(x), x)  # 先按长度的负数（降序），再按词本身（升序）
    )
    # 取前k个，并添加长度信息
    longest_k_words = [(word, len(word)) for word in sorted_by_length[:k]]
    
    return (top_n_words, longest_k_words)
""")

ℹ️ 未追加到文件（开关关闭）


# 关键输出子函数

In [34]:
auto_write('''
def strip_html_tags(text):
    """
    移除文本中的HTML标签并提取纯文本内容，同时标准化换行符。

    功能说明：
        1. 使用BeautifulSoup解析HTML文本，移除`<iframe>`和`<script>`标签（通常包含非文本内容）
        2. 提取HTML中的纯文本内容
        3. 将各种换行符（\\r、\\n、\\r\\n）统一替换为单个\\n，确保换行格式一致

    参数：
        text (str)：包含HTML标签的原始文本字符串

    返回：
        str：移除所有HTML标签后的纯文本，换行符已标准化为\\n

    依赖：
        需要安装BeautifulSoup库（pip install beautifulsoup4）

    示例：
        >>> raw_html = "<p>Hello<br>World!</p><script>alert('test')</script>"
        >>> strip_html_tags(raw_html)
        'Hello\\nWorld!'
    """
    pass
    
def remove_accented_chars(text):
    """
    移除文本中的重音字符（如é、ñ、ü等），将其转换为无重音的基础字符。
    
    处理逻辑：
        1. 使用`unicodedata.normalize('NFKD', text)`对文本进行Unicode规范化：
           - NFKD（Compatibility Decomposition, Canonical Composition）会将重音字符分解为
             基础字符 + 重音符号（例如：'é' 分解为 'e' + 重音符号）。
        2. 通过`.encode('ascii', 'ignore')`将规范化后的文本编码为ASCII：
           - ASCII编码不支持重音符号，`ignore`参数会忽略无法编码的重音符号部分，仅保留基础字符。
        3. 再通过`.decode('utf-8', 'ignore')`将字节流解码回UTF-8字符串，得到无重音的结果。
    
    参数：
        text (str)：包含重音字符的原始文本
    
    返回：
        str：移除重音后的文本（仅包含ASCII可表示的字符）
    
    示例：
        >>> remove_accented_chars("café cliché naïve")
        'cafe cliche naive'
        >>> remove_accented_chars("àèìòù ÁÉÍÓÚ ñ ç")
        'aeiou AEIOU n c'
    """
    pass
    
def pos_tag_wordnet(tagged_tokens):
    """
    将NLTK词性标注结果转换为WordNet词形还原器（Lemmatizer）兼容的词性标签。
    
    背景：
        NLTK的`pos_tag`函数返回的词性标签遵循Penn Treebank格式（如形容词为'JJ'、动词为'VB'等），
        而WordNet的`lemmatize`方法需要特定的词性标签（如形容词为`wordnet.ADJ`、动词为`wordnet.VERB`等），
        因此需要通过首字母映射实现格式转换。
    
    参数：
        tagged_tokens (list)：由`nltk.pos_tag`返回的词性标注列表，每个元素为元组`(word, tag)`，
                             其中`word`是词语，`tag`是Penn Treebank格式的词性标签（如'JJ'、'VB'）。
    
    返回：
        list：转换后的词性标注列表，每个元素为元组`(word, wordnet_tag)`，
              其中`wordnet_tag`是WordNet兼容的词性标签（如`wordnet.ADJ`、`wordnet.VERB`）。
              若标签无法匹配，默认映射为名词（`wordnet.NOUN`）。
    
    映射规则：
        - 'j'（形容词，如Penn标签'JJ'）→ `wordnet.ADJ`
        - 'v'（动词，如Penn标签'VB'）→ `wordnet.VERB`
        - 'n'（名词，如Penn标签'NN'）→ `wordnet.NOUN`
        - 'r'（副词，如Penn标签'RB'）→ `wordnet.ADV`
    
    示例：
        >>> from nltk import pos_tag, word_tokenize
        >>> tagged = pos_tag(word_tokenize("running fast"))  # [('running', 'VBG'), ('fast', 'RB')]
        >>> pos_tag_wordnet(tagged)
        [('running', wordnet.VERB), ('fast', wordnet.ADV)]
    """
    pass


def lemmatize_text(text):
    """
    对文本进行词形还原（Lemmatization），将词语还原为其基本形式（如动词第三人称→原形、名词复数→单数）。
    
    处理流程：
        1. 分词：将文本拆分为独立词语（使用NLTK的`word_tokenize`）。
        2. 词性标注：为每个词语添加Penn Treebank格式的词性标签（使用NLTK的`pos_tag`）。
        3. 标签转换：将词性标签转换为WordNet兼容格式（调用`pos_tag_wordnet`）。
        4. 词形还原：使用WordNet词形还原器，根据转换后的词性标签对每个词语进行还原。
        5. 拼接：将还原后的词语重新拼接为文本。
    
    参数：
        text (str)：待处理的原始文本（英文）。
    
    返回：
        str：经过词形还原后的文本，词语均为基本形式。
    
    依赖：
        - 需要加载NLTK的`punkt`分词模型和`averaged_perceptron_tagger`词性标注模型（可通过`nltk.download()`下载）。
        - 需要初始化WordNet词形还原器（如`wnl = WordNetLemmatizer()`）。
    
    示例：
        >>> wnl = WordNetLemmatizer()  # 假设已初始化
        >>> lemmatize_text("Cats are running quickly")
        'cat be run quick'
    """
    pass       

def remove_special_characters(text, lang='en', remove_digits=False):
    """
    移除文本中的特殊字符（如标点符号、符号等），可选择是否保留数字。
    
    处理逻辑：
        1. 根据`remove_digits`参数构建正则匹配模式:
           - 当`remove_digits=False`（默认）:
             - lang='en': 保留字母(a-zA-Z)、数字(0-9)和空格(\s)，移除其他不相关字符。
             - lang='zh': 保留中文(\u4e00-\u9fa5)、数字(0-9)和空格(\s)，移除其他不相关字符。
           - 当`remove_digits=True`:
             - lang='en': 保留字母(a-zA-Z)、数字(0-9)和空格(\s)，移除其他不相关字符。
             - lang='zh': 保留中文(\u4e00-\u9fa5)、数字(0-9)和空格(\s)，移除其他不相关字符。
        2. 使用`re.sub`将匹配到的特殊字符替换为空字符串，实现移除效果。
        3. 额外处理：移除多余的换行符和空白字符，确保文本整洁。
    
    参数：
        text (str)：待处理的原始文本。
        remove_digits (bool)：是否移除数字，默认False（保留数字）。
        remove_special_characters (bool): 是否移除特殊字符，默认False（保留特殊字符）
    
    返回：
        str：移除特殊字符后的文本（仅保留指定允许的字符）。
    
    示例：
        >>> remove_special_characters("Hello, 世界! 123 🙂🙂🙂", lang='en')
        'Hello  123'
        
        >>> remove_special_characters("Hello, 世界! 123 🙂🙂🙂", lang='zh')
        '世界 123'
        
        >>> remove_special_characters("Hello, 世界! 123 🙂🙂🙂", lang='en', remove_digits=True)
        'Hello '
        
        >>> remove_special_characters("Hello, 世界! 123 🙂🙂🙂", lang='zh', remove_digits=True)
        '世界'    
    """
    pass

def remove_stopwords(text, is_lower_case=False, stopwords=None, lang='en'):
    """
    移除文本中的停用词（如英文的"the"、"is"，中文的"的"、"了"等无实际语义的高频词），支持中英文。
    
    处理流程：
        1. 加载停用词表：若未提供自定义停用词表（stopwords），则根据语言（lang）加载默认停用词表：
           - 英文：使用NLTK的英文停用词表（nltk.corpus.stopwords.words('english')）。
           - 中文：使用NLTK的中文停用词表（nltk.corpus.stopwords.words('chinese')）。
        2. 分词：根据语言选择分词工具：
           - 英文：使用NLTK的`word_tokenize`进行分词。
           - 中文：使用Jieba的`lcut`（精确模式）进行分词。
        3. 过滤停用词：根据`is_lower_case`判断是否需要将词语小写后再匹配停用词表，保留非停用词。
        4. 拼接：将过滤后的词语重新拼接为文本。
    
    参数：
        text (str)：待处理的原始文本。
        is_lower_case (bool)：文本是否已转为小写，默认False（需将词语小写后再匹配停用词）。
        stopwords (list, optional)：自定义停用词表，若为None则使用默认表。
        lang (str)：语言类型，'en'（英文）或'zh'（中文），默认'en'。
    
    返回：
        str：切词并移除停用词后用空格相连的文本。
    
    依赖：
        - 英文：需加载NLTK的`punkt`分词模型（nltk.download('punkt')）和英文停用词表（nltk.download('stopwords')）。
        - 中文：需安装Jieba（pip install jieba）和加载NLTK的中文停用词表（nltk.download('stopwords')）。
    
    示例：
        >>> # 英文示例
        >>> remove_stopwords("The quick brown fox jumps over the lazy dog", lang='en')
        'quick brown fox jumps lazy dog'
        >>> # 中文示例
        >>> remove_stopwords("这只敏捷的棕色狐狸跳过了那只懒狗", lang='zh')
        '只 敏捷 棕色 狐狸 跳过 只 懒 狗'
    """
    pass    
''')

ℹ️ 未追加到文件（开关关闭）


# 关键输出主函数

In [35]:
auto_write(''' 
def normalize_doc(doc, 
                     html_stripping=True, 
                     contraction_expansion=True,
                     accented_char_removal=True, 
                     text_lower_case=True,
                     text_lemmatization=True, 
                     special_char_removal=True,
                     stopword_removal=True, 
                     remove_digits=False, # Default: keep digits
                     zh_simplification=True,
                     isDebug=False):
    """
    规范化文本语料库，支持多种预处理操作，包括HTML标签移除、缩写扩展、重音字符移除、
    小写转换、词形还原、特殊字符移除、停用词移除等，适用于中英文文本。
    参数：
        doc (str): 输入文档。
        html_stripping (bool): 是否移除HTML标签，默认True。
        contraction_expansion (bool): 是否扩展缩写，默认True。
        accented_char_removal (bool): 是否移除重音字符，默认True。
        text_lower_case (bool): 是否将文本转换为小写，默认True。
        text_lemmatization (bool): 是否进行词形还原，默认True。
        special_char_removal (bool): 是否移除特殊字符，默认True。
        stopword_removal (bool): 是否移除停用词，默认True。
        remove_digits (bool): 是否移除数字，默认False。
        zh_simplification (bool): 是否将中文繁体转换为简体，默认True。
        isDebug (bool): 是否打印调试信息，默认False。
    返回：
        元组 `(doc, lang)`，其中：
            - doc (str)：预处理后的文本
            - lang (str)：文本语言（'en'/'zh'/'unknown'）    
    """                              
    pass
''') 

ℹ️ 未追加到文件（开关关闭）


# 实现关键子函数

In [36]:
def strip_html_tags(text):
    """
    移除文本中的HTML标签并提取纯文本内容，同时标准化换行符。

    功能说明：
        1. 使用BeautifulSoup解析HTML文本，移除`<iframe>`和`<script>`标签（通常包含非文本内容）
        2. 提取HTML中的纯文本内容
        3. 将各种换行符（\r、\n、\r\n）统一替换为单个\n，确保换行格式一致

    参数：
        text (str)：包含HTML标签的原始文本字符串

    返回：
        str：移除所有HTML标签后的纯文本，换行符已标准化为\n

    依赖：
        需要安装BeautifulSoup库（pip install beautifulsoup4）

    示例：
        >>> raw_html = "<p>Hello<br>World!</p><script>alert('test')</script>"
        >>> strip_html_tags(raw_html)
        'Hello\\nWorld!'
    """
    soup = BeautifulSoup(text, "html.parser")
    [s.extract() for s in soup(['iframe', 'script'])]
    stripped_text = soup.get_text()
    stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
    
    return stripped_text

def remove_accented_chars(text):
    """
     (NFKD) will apply the compatibility decomposition, i.e.
     replace all compatibility characters with their equivalents.
    """
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

def pos_tag_wordnet(tagged_tokens):
    """
    将NLTK词性标注结果转换为WordNet词形还原器（Lemmatizer）兼容的词性标签。
    
    背景：
        NLTK的`pos_tag`函数返回的词性标签遵循Penn Treebank格式（如形容词为'JJ'、动词为'VB'等），
        而WordNet的`lemmatize`方法需要特定的词性标签（如形容词为`wordnet.ADJ`、动词为`wordnet.VERB`等），
        因此需要通过首字母映射实现格式转换。
    
    参数：
        tagged_tokens (list)：由`nltk.pos_tag`返回的词性标注列表，每个元素为元组`(word, tag)`，
                             其中`word`是词语，`tag`是Penn Treebank格式的词性标签（如'JJ'、'VB'）。
    
    返回：
        list：转换后的词性标注列表，每个元素为元组`(word, wordnet_tag)`，
              其中`wordnet_tag`是WordNet兼容的词性标签（如`wordnet.ADJ`、`wordnet.VERB`）。
              若标签无法匹配，默认映射为名词（`wordnet.NOUN`）。
    
    映射规则：
        - 'j'（形容词，如Penn标签'JJ'）→ `wordnet.ADJ`
        - 'v'（动词，如Penn标签'VB'）→ `wordnet.VERB`
        - 'n'（名词，如Penn标签'NN'）→ `wordnet.NOUN`
        - 'r'（副词，如Penn标签'RB'）→ `wordnet.ADV`
    
    示例：
        >>> from nltk import pos_tag, word_tokenize
        >>> tagged = pos_tag(word_tokenize("running fast"))  # [('running', 'VBG'), ('fast', 'RB')]
        >>> pos_tag_wordnet(tagged)
        [('running', wordnet.VERB), ('fast', wordnet.ADV)]
    """
    # 定义Penn Treebank标签首字母到WordNet标签的映射
    tag_map = {'j': wordnet.ADJ, 'v': wordnet.VERB, 'n': wordnet.NOUN, 'r': wordnet.ADV}
    # 转换标签：取原标签首字母小写，查映射表；未匹配则默认名词
    new_tagged_tokens = [(word, tag_map.get(tag[0].lower(), wordnet.NOUN))
                            for word, tag in tagged_tokens]
    return new_tagged_tokens

def lemmatize_text(text):
    """
    对文本进行词形还原（Lemmatization），将词语还原为其基本形式（如动词第三人称→原形、名词复数→单数）。
    
    处理流程：
        1. 分词：将文本拆分为独立词语（使用NLTK的`word_tokenize`）。
        2. 词性标注：为每个词语添加Penn Treebank格式的词性标签（使用NLTK的`pos_tag`）。
        3. 标签转换：将词性标签转换为WordNet兼容格式（调用`pos_tag_wordnet`）。
        4. 词形还原：使用WordNet词形还原器，根据转换后的词性标签对每个词语进行还原。
        5. 拼接：将还原后的词语重新拼接为文本。
    
    参数：
        text (str)：待处理的原始文本（英文）。
    
    返回：
        str：经过词形还原后的文本，词语均为基本形式。
    
    依赖：
        - 需要加载NLTK的`punkt`分词模型和`averaged_perceptron_tagger`词性标注模型（可通过`nltk.download()`下载）。
        - 需要初始化WordNet词形还原器（如`wnl = WordNetLemmatizer()`）。
    
    示例：
        >>> wnl = WordNetLemmatizer()  # 假设已初始化
        >>> lemmatize_text("Cats are running quickly")
        'cat be run quick'
    """
    # 1. 分词：将文本拆分为词语列表
    tokens = nltk.word_tokenize(text)
    # 2. 词性标注：为每个词语添加Penn Treebank格式标签
    tagged_tokens = nltk.pos_tag(tokens)
    # 3. 转换标签：适配WordNet词形还原器
    wordnet_tokens = pos_tag_wordnet(tagged_tokens)
    # 4. 词形还原：根据词性标签还原每个词语，再拼接为文本
    text = ' '.join(wnl.lemmatize(word, tag) for word, tag in wordnet_tokens)
    return text

def remove_special_characters(text, lang='en', remove_digits=False):
    """
    移除文本中的特殊字符（如标点符号、符号等），可选择是否保留数字。
    
    处理逻辑：
        1. 根据`remove_digits`参数构建正则匹配模式:
           - 当`remove_digits=False`（默认）:
             - lang='en': 保留字母(a-zA-Z)、数字(0-9)和空格(\s)，移除其他不相关字符。
             - lang='zh': 保留中文(\\u4e00-\\u9fa5)、数字(0-9)和空格(\\s)，移除其他不相关字符。
           - 当`remove_digits=True`:
             - lang='en': 保留字母(a-zA-Z)、数字(0-9)和空格(\s)，移除其他不相关字符。
             - lang='zh': 保留中文(\\u4e00-\\u9fa5)、数字(0-9)和空格(\\s)，移除其他不相关字符。
        2. 使用`re.sub`将匹配到的特殊字符替换为空字符串，实现移除效果。
        3. 额外处理：移除多余的换行符和空白字符，确保文本整洁。
    
    参数：
        text (str)：待处理的原始文本。
        remove_digits (bool)：是否移除数字，默认False（保留数字）。
        remove_special_characters (bool): 是否移除特殊字符，默认False（保留特殊字符）
    
    返回：
        str：移除特殊字符后的文本（仅保留指定允许的字符）。
    
    示例：
        >>> remove_special_characters("Hello, 世界! 123 🙂🙂🙂", lang='en')
        'Hello  123'
        
        >>> remove_special_characters("Hello, 世界! 123 🙂🙂🙂", lang='zh')
        '世界 123'
        
        >>> remove_special_characters("Hello, 世界! 123 🙂🙂🙂", lang='en', remove_digits=True)
        'Hello '
        
        >>> remove_special_characters("Hello, 世界! 123 🙂🙂🙂", lang='zh', remove_digits=True)
        '世界'    
    """
    if not text:
        return ""
    
    # 构建正则模式
    if lang == 'en':
        if remove_digits:
            # 英文模式：保留字母和空格，移除数字
            pattern = r'[^a-zA-Z\s]'
        else:
            # 英文模式：保留字母、数字和空格
            pattern = r'[^a-zA-Z0-9\s]'
    elif lang == 'zh':
        if remove_digits:
            # 中文模式：保留中文字符和空格，移除数字
            pattern = r'[^\u4e00-\u9fa5\s]'
        else:
            # 中文模式：保留中文、数字和空格
            pattern = r'[^\u4e00-\u9fa50-9\s]'
    else:
        raise ValueError("不支持的语言类型，仅支持 'en' 或 'zh'")
    
    # 移除特殊字符
    text = re.sub(pattern, '', text)

    # 匹配其中包含的任意一个字符。具体包含的字符是：{、.、(、-、)、!、}    
    special_char_pattern = re.compile(r'([{.(-)!}])')
    
    # 将匹配到的字符替换为 空格 + 捕获到的字符 + 空格
    # insert spaces between special characters to isolate them
    text = special_char_pattern.sub(" \\1 ", text)

    # remove extra newlines
    text = re.sub(r'[\r|\n|\r\n]+', ' ', text)

    # remove extra whitespace
    text = re.sub(' +', ' ', text)

    return text

def remove_stopwords(text, is_lower_case=False, stopwords=None, lang='en')->str:
    """
    移除文本中的停用词（如英文的"the"、"is"，中文的"的"、"了"等无实际语义的高频词），支持中英文。
    
    处理流程：
        1. 加载停用词表：若未提供自定义停用词表（stopwords），则根据语言（lang）加载默认停用词表：
           - 英文：使用NLTK的英文停用词表（nltk.corpus.stopwords.words('english')）。
           - 中文：使用NLTK的中文停用词表（nltk.corpus.stopwords.words('chinese')）。
        2. 分词：根据语言选择分词工具：
           - 英文：使用NLTK的`word_tokenize`进行分词。
           - 中文：使用Jieba的`lcut`（精确模式）进行分词。
        3. 过滤停用词：根据`is_lower_case`判断是否需要将词语小写后再匹配停用词表，保留非停用词。
        4. 拼接：将过滤后的词语重新拼接为文本。
    
    参数：
        text (str)：待处理的原始文本。
        is_lower_case (bool)：文本是否已转为小写，默认False（需将词语小写后再匹配停用词）。
        stopwords (list, optional)：自定义停用词表，若为None则使用默认表。
        lang (str)：语言类型，'en'（英文）或'zh'（中文），默认'en'。
    
    返回：
        str：切词并移除停用词后用空格相连的文本。
    
    依赖：
        - 英文：需加载NLTK的`punkt`分词模型（nltk.download('punkt')）和英文停用词表（nltk.download('stopwords')）。
        - 中文：需安装Jieba（pip install jieba）和加载NLTK的中文停用词表（nltk.download('stopwords')）。
    
    示例：
        >>> # 英文示例
        >>> remove_stopwords("The quick brown fox jumps over the lazy dog", lang='en')
        'quick brown fox jumps lazy dog'
        >>> # 中文示例
        >>> remove_stopwords("这只敏捷的棕色狐狸跳过了那只懒狗", lang='zh')
        '只 敏捷 棕色 狐狸 跳过 只 懒 狗'
    """
    # 若未提供自定义停用词表，则加载对应语言的默认停用词表
    if not stopwords:
        if lang == 'en':
            stopwords = nltk.corpus.stopwords.words('english')
        elif lang == 'zh':
            stopwords = nltk.corpus.stopwords.words('chinese')
        else:
            raise ValueError("Unsupported language. Use 'en' for English or 'zh' for Chinese.")   

    if lang == 'en':
        tokens = nltk.word_tokenize(text)
    elif lang == 'zh':
        tokens = jieba.lcut(text)  # 精确模式，返回列表
    else:
        raise ValueError("Unsupported language. Use 'en' for English or 'zh' for Chinese.")

    # 过滤停用词：根据文本是否小写，决定是否将词语转为小写后匹配
    if lang == 'en':
        if is_lower_case:
            # 文本已小写，直接匹配停用词表
            filtered_tokens = [token for token in tokens if token not in stopwords]
        else:
            # 文本未小写，将词语转为小写后再匹配（避免因大小写导致漏判）
            filtered_tokens = [token for token in tokens if token.lower() not in stopwords]   
    elif lang == 'zh':
        filtered_tokens = [token for token in tokens if token not in stopwords]    
                
    # 将过滤后的词语拼接为文本
    filtered_text = ' '.join(filtered_tokens)    

    return filtered_text

# 测试关键子函数

In [37]:
raw_html = "<p>Hello<br>World!</p><script>alert('test')</script>"
print(strip_html_tags(raw_html))

print(remove_accented_chars("café cliché naïve"))
print(remove_accented_chars("àèìòù ÁÉÍÓÚ ñ ç"))

tagged = nltk.pos_tag(nltk.word_tokenize("running fast"))  # [('running', 'VBG'), ('fast', 'RB')]
print(pos_tag_wordnet(tagged))
print(lemmatize_text("Cats are running quickly"))
        
print(remove_special_characters("Hello, World! 123", remove_digits=False))
print(remove_special_characters("Hello, World! 123", remove_digits=True))

HelloWorld!
cafe cliche naive
aeiou AEIOU n c
[('running', 'v'), ('fast', 'r')]
Cats be run quickly
Hello World 123
Hello World 


# 实现关键函数，调用关键子函数

In [38]:
def normalize_doc(doc, 
                     html_stripping=True, 
                     contraction_expansion=True,
                     accented_char_removal=True, 
                     text_lower_case=True,
                     text_lemmatization=True, 
                     special_char_removal=True,
                     stopword_removal=True, 
                     remove_digits=False, # Default: keep digits
                     zh_simplification=True,
                     isDebug=False):
    """
    规范化文本语料库，支持多种预处理操作，包括HTML标签移除、缩写扩展、重音字符移除、
    小写转换、词形还原、特殊字符移除、停用词移除等，适用于中英文文本。
    参数：
        doc (str): 输入文档。
        html_stripping (bool): 是否移除HTML标签，默认True。
        contraction_expansion (bool): 是否扩展缩写，默认True。
        accented_char_removal (bool): 是否移除重音字符，默认True。
        text_lower_case (bool): 是否将文本转换为小写，默认True。
        text_lemmatization (bool): 是否进行词形还原，默认True。
        special_char_removal (bool): 是否移除特殊字符，默认True。
        stopword_removal (bool): 是否移除停用词，默认True。
        remove_digits (bool): 是否移除数字，默认False。
        zh_simplification (bool): 是否将中文繁体转换为简体，默认True。
        isDebug (bool): 是否打印调试信息，默认False。
    返回：
        元组 `(doc, lang)`，其中：
            - doc (str)：预处理后的文本
            - lang (str)：文本语言（'en'/'zh'/'unknown'）    
    """          
    # normalize the document
    if not doc:  # 跳过空文档
        print("警告：输入文档为空，跳过预处理。")
        return doc, 'unknown'
        
    # 1. 检测语言
    lang = detect_language(doc)

    if isDebug:
        print(f"检测到的语言: {lang}")
    
    if lang not in ['en', 'zh']:
        print(f"警告：无法识别文档语言，跳过预处理：{doc[:50]}...\n{doc[len(doc)-50:]}")
    
    # strip HTML
    if html_stripping:
        doc = strip_html_tags(doc)
        if isDebug:
            print(f"HTML标签移除后: {doc[:50]}...\n{doc[len(doc)-50:]}")
        
    if lang == 'en':
        # remove accented characters
        if accented_char_removal:
            doc = remove_accented_chars(doc)

            if isDebug:
                print(f"重音字符移除后: {doc[:50]}...\n{doc[len(doc)-50:]}")
    elif lang == 'zh':
        # Chinese text specific processing
        # 繁体转简体
        if zh_simplification:
            doc = cc_zh.convert(doc)
            if isDebug: 
                print(f"中文繁体转换为简体: {doc[:50]}...\n{doc[len(doc)-50:]}")
    else:
        # 未知语言，跳过后续处理
        print("警告：无法识别文档语言，跳过预处理")
        return doc, 'unknown'
    
    # expand contractions
    if contraction_expansion:
        doc = contractions.fix(doc)

        if isDebug:
            print(f"缩写扩展后: {doc[:50]}...\n{doc[len(doc)-50:]}")
        
    # lowercase the text
    if text_lower_case:
        doc = doc.lower()

        if isDebug:
            print(f"小写转换后: {doc[:50]}...\n{doc[len(doc)-50:]}")
        
        # lemmatize text
        if text_lemmatization:
            doc = lemmatize_text(doc)
            if isDebug:
                print(f"词形还原后: {doc[:50]}...\n{doc[len(doc)-50:]}")

    # remove special characters and\or digits
    if special_char_removal:
        doc = remove_special_characters(doc, lang, remove_digits=remove_digits)
        
        if isDebug:
            print(f"特殊字符移除后: {doc[:50]}...\n{doc[len(doc)-50:]}")

    # remove stopwords
    if stopword_removal:
        doc = remove_stopwords(doc, is_lower_case=text_lower_case, lang=lang)
        
        if isDebug:
            print(f"停用词移除后: {doc[:50]}...\n{doc[len(doc)-50:]}")
                
    return doc, lang 


# 获取文本内容

In [39]:
auto_write('''
def fetch_gutenberg_text(file_path=None):
    """
    读取本地古腾堡txt文件，提取正文内容（去除首尾元数据标记）
    
    参数：
        file_path (str): 本地txt文件路径，默认 './data/1342-0.txt'
    
    返回：
        str: 提取的正文内容；若文件读取失败/无有效标记，返回None
    
    示例：
        >>> content = fetch_gutenberg_text('./data/1342-0.txt')
        >>> print(content[:500])  # 打印正文前500字符
           
    说明: 
        你可以自己实现输入 fetch_gutenberg_text(link='https://www.gutenberg.org/files/1342/1342-0.txt'):
        >>> content = fetch_gutenberg_text('https://www.gutenberg.org/files/1342/1342-0.txt')
        >>> print(content[:500])  # 打印正文前500字符
    """
    # 1. 验证文件格式（仅支持txt）
    if not file_path.endswith('.txt'):
        print(f"错误：{file_path} 不是.txt格式文件，仅支持文本文件")
        return None

    # 2. 正则匹配古腾堡标准标记（提取正文核心）
    # 匹配 "*** START OF THE PROJECT GUTENBERG EBOOK ... ***" 及变体
    start_pat = re.compile(
        r'\*{3}\s+START OF (?:THE|THIS)?\s+PROJECT GUTENBERG EBOOK.*?\*{3}',
        re.IGNORECASE | re.DOTALL
    )
    # 匹配 "*** END OF THE PROJECT GUTENBERG EBOOK ... ***" 及变体
    end_pat = re.compile(
        r'\*{3}\s+END OF (?:THE|THIS)?\s+PROJECT GUTENBERG EBOOK.*?\*{3}',
        re.IGNORECASE | re.DOTALL
    )

    # 3. 读取本地文件（适配古腾堡常见编码：utf-8 / latin-1）
    try:
        print(f"正在读取本地文件：{file_path}")
        
        # 优先尝试utf-8编码（古腾堡现代文本常用）
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                text = f.read()
            print("文件编码：utf-8（读取成功）")
        
        # 若utf-8解码失败，尝试latin-1（古腾堡早期英文文本常用）
        except UnicodeDecodeError:
            with open(file_path, 'r', encoding='latin-1') as f:
                text = f.read()
            print("文件编码：latin-1（读取成功）")

    # 捕获本地文件读取的常见错误
    except FileNotFoundError:
        print(f"错误：文件 {file_path} 不存在，请检查路径是否正确")
        return None
    except PermissionError:
        print(f"错误：无权限读取文件 {file_path}，请检查文件权限")
        return None
    except Exception as e:
        print(f"错误：读取文件时发生未知错误 - {str(e)[:50]}...")
        return None

    # 4. 提取正文（基于古腾堡标记）
    start_match = start_pat.search(text)
    end_match = end_pat.search(text)

    # 处理标记不存在/位置异常的情况
    if not start_match:
        print("警告：未找到古腾堡正文开始标记（*** START OF ... ***）")
        # 可选：返回全文（若用户希望保留完整文件内容）
        # return text.strip()
        return None
    if not end_match:
        print("警告：未找到古腾堡正文结束标记（*** END OF ... ***）")
        # 可选：返回开始标记后的内容
        # return text[start_match.end():].strip()
        return None
    if start_match.end() >= end_match.start():
        print("错误：开始标记位置在结束标记之后，无法提取正文")
        return None

    # 提取标记间的正文（去除首尾空格）
    content = text[start_match.end():end_match.start()].strip()
    print(f"正文提取成功！正文长度：{len(content)} 字符")
    return content
''')

ℹ️ 未追加到文件（开关关闭）


In [40]:
auto_write('''
from pathlib import Path  # 确保已导入

def save_processed_text(original_path, processed_text):
    """
    将预处理后的文本保存至新文件，命名规则：原文件xxx.txt → xxx-p.txt
    
    参数：
        original_path (str): 原文件路径（如 './data/1342-0.txt'）
        processed_text (str): 预处理后的文本内容
    """
    # 转换为Path对象，方便处理路径
    orig_path = Path(original_path)
    
    # 生成新文件名：原文件名（不含扩展名）+ "-p" + 扩展名
    new_filename = f"{orig_path.stem}-p{orig_path.suffix}"  # 如 "1342-0-p.txt"
    
    # 构建新文件的完整路径（与原文件同目录）
    new_path = orig_path.parent / new_filename  # 如 './data/1342-0-p.txt'
    
    try:
        # 确保目标目录存在（若不存在则创建）
        new_path.parent.mkdir(parents=True, exist_ok=True)
        
        # 写入预处理后的文本（使用utf-8编码，避免中文乱码）
        with open(new_path, 'w', encoding='utf-8') as f:
            f.write(processed_text)
        
        print(f"✅ 预处理文本已保存至：{new_path}")
    except Exception as e:
        print(f"❌ 保存文件失败 {new_path}：{str(e)}")
''')

ℹ️ 未追加到文件（开关关闭）


In [41]:
auto_write('''
def test_english_contractions(processed: str)->list:
    """
    检查处理后的文本中是否仍包含未扩展的英文缩写格式
    
    参数：
        processed: 处理后的文本
    返回：
        错误列表（包含仍存在的缩写格式）
    """
    errors = []
    # 统一转为小写检查（忽略大小写影响）
    processed_lower = processed.lower()
    
    # 遍历contractions包支持的所有缩写
    for contraction in contractions.contractions_dict.keys():
        # 1. 检查处理后的文本中是否存在该缩写（完整单词匹配）
        # 正则：\b 确保匹配独立单词，re.escape处理特殊字符（如撇号'）
        contraction_pattern = re.compile(rf'\b{re.escape(contraction)}\b')
        if contraction_pattern.search(processed_lower):
            # 若存在未扩展的缩写，记录错误
            errors.append(f"处理后的文本仍包含未扩展的缩写: '{contraction}'")
    
    return errors

def test_stopwords(processed:str, lang='en')->list:
    """
    假设processed是已经切词用空格分隔连接的字符串
    """
    errors = []

    if lang == 'en':
        stopwords_en = set(nltk.corpus.stopwords.words('english'))
    elif lang == 'zh':
        stopwords_zh = set(nltk.corpus.stopwords.words('chinese'))
    else: 
        raise ValueError("Unsupported language. Use 'en' or 'zh'.")
            
    # tokens = nltk.word_tokenize(processed) if lang == 'en' else jieba.lcut(processed)
    tokens = processed.split()

    if lang == 'en' and any(token in stopwords_en for token in tokens):
        errors.append("英文停用词未移除")
    if lang == 'zh' and any(token in stopwords_zh for token in tokens):
        errors.append("中文停用词未移除")
        
    return errors

def test_preprocessed_text(processed, lang)->list:
    """验证预处理结果"""
    errors = []
    # 检查HTML标签是否移除（假设原始文本含HTML标签，此处简化为检查特殊标签字符）
    if '<' in processed or '>' in processed:
        errors.append("HTML标签未完全移除")
    
    # 检查小写转换
    if any(c.isupper() for c in processed) and lang == 'en':
        errors.append("英文文本未转为小写")
    
    # 检查英文缩写扩展（示例：don't → do not）
    if lang == 'en':
        err_test_english_contractions = test_english_contractions(processed) 
        if len(err_test_english_contractions)>0: # not empty
            errors.extend(err_test_english_contractions)
    
    # 检查停用词移除（英文示例：'the'；中文示例：'的'）
    err_test_stopwords = test_stopwords(processed, lang)
    if len(err_test_stopwords)>0:
        errors.extend(err_test_stopwords)

    # 检查特殊字符移除（示例：'!' '?' 等）
    if any(c in '!@#$%^&*()_+' for c in processed):
        errors.append("特殊字符未移除")
    
    return errors
''')

ℹ️ 未追加到文件（开关关闭）


# 测试获取文本


In [ ]:
# 1. 获取测试数据
print("=== 获取测试数据 ===")

booknames = ['PRIDE and PREJUDICE', 'A Dream Of Red Mansions', 'Romance of the Three Kingdoms']
file_paths = ['./data/1342-0.txt', './data/24264-0.txt', './data/23950-0.txt']

docs = []
processed_docs = []
langs = []

for i, file_path in enumerate(file_paths):
    ori_doc = fetch_gutenberg_text(file_path)
    if ori_doc:
        docs.append(ori_doc)

    processed_doc, lang = normalize_doc(ori_doc, isDebug=False)
    save_processed_text(file_path, processed_doc)
    processed_docs.append(processed_doc)
    langs.append(lang)

    bookname = booknames[i]
    n, k = 10, 10  # 统计前n高频词和前k长词


    print(f"\n=== 书名: {bookname} ===")
    print(f"\n=== 处理后文本（语言：{lang}）===\n{processed_doc[:100]}...\n文本长度：{len(processed_doc)}字符")
    top_n_words, longest_k_words = get_statistics(processed_doc, n, k, lang=lang)
    print(f"文本前10高频词：{top_n_words}")
    print(f"文本前10长词：{longest_k_words}")

    test_errors = test_preprocessed_text(processed_doc, lang)
    if not test_errors:
        print("✅ 所有检查通过")
    else:
        print("❌ 错误：")
        for e in test_errors:
            print(f"- {e}")

=== 获取测试数据 ===
正在读取本地文件：./data/1342-0.txt
文件编码：utf-8（读取成功）
正文提取成功！正文长度：728713 字符
✅ 预处理文本已保存至：data/1342-0-p.txt

=== 书名: PRIDE and PREJUDICE ===

=== 处理后文本（语言：en）===
illustration george allen publisher 156 char cross road london ruskin house illustration reading jan...
文本长度：401496字符
文本前10高频词：[('mr', 809), ('elizabeth', 614), ('say', 614), ('could', 531), ('would', 486), ('know', 391), ('darcy', 385), ('mrs', 353), ('bennet', 349), ('make', 343)]
文本前10长词：[('charactercreation', 17), ('communicativeness', 17), ('disinterestedness', 17), ('eighteenthcentury', 17), ('fordycebelectured', 17), ('misrepresentation', 17), ('agreeablelooking', 16), ('discontentedness', 16), ('fellowtravellers', 16), ('greatgrandmother', 16)]
✅ 所有检查通过
正在读取本地文件：./data/24264-0.txt
文件编码：utf-8（读取成功）
正文提取成功！正文长度：906088 字符


Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/6_/9scks0f16w9ck73gbx9wznbh0000gn/T/jieba.cache
Loading model cost 0.272 seconds.
Prefix dict has been built successfully.


✅ 预处理文本已保存至：data/24264-0-p.txt

=== 书名: A Dream Of Red Mansions ===

=== 处理后文本（语言：zh）===
第一回   甄士隐 梦幻 识通灵   贾雨村 风尘 怀 闺秀   开卷 第一回 作者 自云因 曾 历过 一番 梦幻 真事 隐去   借   通灵   说 撰此 石头记 一书 故曰   甄士隐   但书...
文本长度：947985字符
文本前10高频词：[('道', 6491), ('说', 6111), ('不', 4107), ('去', 3864), ('宝玉', 3618), ('著', 3175), ('便', 3173), ('人', 3005), ('都', 2614), ('笑', 2487)]
文本前10长词：[('一个巴掌拍不响', 7), ('不是冤家不聚头', 7), ('人怕出名猪怕壮', 7), ('千里姻缘一线牵', 7), ('得饶人处且饶人', 7), ('心有余而力不足', 7), ('情人眼里出西施', 7), ('敢把皇帝拉下马', 7), ('机关算尽太聪明', 7), ('知人知面不知心', 7)]
✅ 所有检查通过
正在读取本地文件：./data/23950-0.txt
文件编码：utf-8（读取成功）
正文提取成功！正文长度：622115 字符
✅ 预处理文本已保存至：data/23950-0-p.txt

=== 书名: Romance of the Three Kingdoms ===

=== 处理后文本（语言：zh）===
  第一回 宴 桃园 豪杰 三 结义 斩 黄巾 英雄 首 立功   词曰   滚滚 长江 东 逝水 浪花 淘尽 英雄 是非成败 转头 空 青山 依旧 几度 夕阳红 白   发 渔樵 江渚 上惯 看 秋...
文本长度：703814字符
文本前10高频词：[('曰', 7700), ('吾', 1827), ('不', 1630), ('去', 1226), ('皆', 1131), ('人', 1064), ('见', 952), ('曹操', 911), ('遂', 828), ('孔明', 802)]
文本前10长词：[('三分天下有其二', 7), ('勿以恶小而为之', 7), ('小不忍则乱大谋', 7),

In [43]:
# 测试检测代码
def load_file(file_path:str)->str: 
    try:
        print(f"正在读取本地文件：{file_path}")
        
        # 优先尝试utf-8编码（古腾堡现代文本常用）
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                text = f.read()
            print("文件编码：utf-8（读取成功）")
        
        # 若utf-8解码失败，尝试latin-1（古腾堡早期英文文本常用）
        except UnicodeDecodeError:
            with open(file_path, 'r', encoding='latin-1') as f:
                text = f.read()
            print("文件编码：latin-1（读取成功）")

    # 捕获本地文件读取的常见错误
    except FileNotFoundError:
        print(f"错误：文件 {file_path} 不存在，请检查路径是否正确")
        return None
    except PermissionError:
        print(f"错误：无权限读取文件 {file_path}，请检查文件权限")
        return None
    except Exception as e:
        print(f"错误：读取文件时发生未知错误 - {str(e)[:50]}...")
        return None

    return text

# 假设你已经有了 filtered_tokens 列表（例如从 remove_stopwords 函数中获取）
# 示例：filtered_tokens = ['敏捷', '棕色', '狐狸', '跳过', '懒狗']

def save_tokens(filtered_tokens, file_path="filtered_tokens.txt"):
    """
    将过滤后的 tokens 保存到文本文件，每行一个 token
    
    参数：
        filtered_tokens (list): 过滤后的词语列表
        file_path (str): 保存的文件路径（默认当前目录下的 filtered_tokens.txt）
    """
    try:
        # 打开文件并写入，每行一个 token
        with open(file_path, 'w', encoding='utf-8') as f:
            for token in filtered_tokens:
                f.write(f"{token}\n")  # 每行写一个 token，便于查看
        print(f"✅ filtered_tokens 已保存到：{file_path}")
    except Exception as e:
        print(f"❌ 保存失败：{str(e)}")

In [ ]:
#ori_doc = load_file('./data/1342-0.txt')
# processed_doc, lang = normalize_doc(ori_doc, isDebug=False)
#processed_doc2 = load_file('./data/1342-0-p.txt')
#lang = 'en'

ori_doc = load_file('./data/23950-0.txt')

test_v3 = test_stopwords(ori_doc, 'zh')
print(f'v3: {test_v3}')

正在读取本地文件：./data/23950-0.txt
文件编码：utf-8（读取成功）
v3: []


In [45]:
print(ori_doc[:1000])
'''
processed_doc = cc_zh.convert(ori_doc)
processed_doc = remove_special_characters(processed_doc, lang='zh')
print(processed_doc[:1000])
processed_doc = remove_stopwords(processed_doc, lang='zh')
'''

processed_doc, lang = normalize_doc(ori_doc, isDebug=False)
test_v4 = test_stopwords(processed_doc, 'zh')

print(f'v4: {test_v4}')


Project Gutenberg's Romance of the Three Kingdoms, by Guanzhong Luo

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org


Title: Romance of the Three Kingdoms

Author: Guanzhong Luo

Release Date: December 21, 2007 [EBook #23950]

Language: Chinese

Character set encoding: UTF-8

*** START OF THIS PROJECT GUTENBERG EBOOK ROMANCE OF THE THREE KINGDOMS ***




Produced by Jian-Lun Huang




第一回：宴桃園豪傑三結義，斬黃巾英雄首立功

　　詞曰：

　　滾滾長江東逝水，浪花淘盡英雄。是非成敗轉頭空：青山依舊在，幾度夕陽紅。白
髮漁樵江渚上，慣看秋月春風。一壺濁酒喜相逢：古今多少事，都付笑談中。

　　話說天下大勢，分久必合，合久必分：周末七國分爭，并入於秦。及秦滅之後，楚
、漢分爭，又并入於漢。漢朝自高祖斬白蛇而起義，一統天下。後來光武中興，傳至獻
帝，遂分為三國。推其致亂之由，殆始於桓、靈二帝。桓帝禁錮善類，崇信宦官。及桓
帝崩，靈帝即位，大將軍竇武、太傅陳蕃，共相輔佐。時有宦官曹節等弄權，竇武、陳
蕃謀誅之，作事不密，反為所害。中涓自此愈橫。

　　建寧二年四月望日，帝御溫德殿。方陞座，殿角狂風驟起，只見一條大青蛇，從梁
上飛將下來，蟠於椅上。帝驚倒，左右急救入宮，百官俱奔避。須臾，蛇不見了。忽然
大雷大雨，加以冰雹，落到半夜方止，壞卻房屋無數。建寧四年二月，洛陽地震；又海
水泛溢，沿海居


In [46]:
import os

def find_stopwords_in_file(token_file_path, lang='en'):
    """
    从 token 文件中找出所有属于停用词列表的 token
    
    参数：
        token_file_path (str): token 文件路径（如 'tmp_tokens.txt'）
        lang (str): 语言，'en' 为英文，'zh' 为中文（默认英文）
    返回：
        list: 属于停用词的 token 列表
    """
    # 2. 加载对应语言的停用词列表（转为集合，提高查询效率）
    if lang == 'en':
        stopword_set = set(nltk.corpus.stopwords.words('english'))
    elif lang == 'zh':
        stopword_set = set(nltk.corpus.stopwords.words('chinese'))
    else:
        raise ValueError("不支持的语言，仅支持 'en'（英文）或 'zh'（中文）")
    
    # 3. 读取 token 文件（每行一个 token）
    if not os.path.exists(token_file_path):
        raise FileNotFoundError(f"文件 {token_file_path} 不存在")
    
    with open(token_file_path, 'r', encoding='utf-8') as f:
        # 读取所有行，去除每行的换行符和前后空格
        tokens = [line.strip() for line in f.readlines() if line.strip()]  # 过滤空行
    
    # 4. 找出属于停用词的 token
    stopwords_in_file = [token for token in tokens if token in stopword_set]
    
    # 5. 输出结果
    print(f"在 {token_file_path} 中找到 {len(stopwords_in_file)} 个停用词：")
    for idx, sw in enumerate(stopwords_in_file, 1):
        print(f"{idx}. {sw}")
    
    # 可选：保存结果到文件
    output_file = f"stopwords_in_{os.path.basename(token_file_path)}"
    with open(output_file, 'w', encoding='utf-8') as f:
        for sw in stopwords_in_file:
            f.write(f"{sw}\n")
    print(f"\n结果已保存到：{output_file}")
    
    return stopwords_in_file

In [47]:
token_file_path = 'tmp_token.txt'
save_file = find_stopwords_in_file(token_file_path, lang='zh') 

在 tmp_token.txt 中找到 0 个停用词：

结果已保存到：stopwords_in_tmp_token.txt


In [48]:
def test_all(doc, lang='en'): 
    doc_ori = doc

    print(f"File1: {doc[:100]}...\n{doc[len(doc)-100:]}")
    doc = strip_html_tags(doc) 
    print(f"After strip_html_tags: {doc[:100]}\n{doc[len(doc)-100:]}")

    if lang == 'en':
        doc = remove_accented_chars(doc)
        print(f"After remove_accented_chars: {doc[:100]}\n{doc[len(doc)-100:]}")
    elif lang == 'zh':
        doc = cc_zh.convert(doc)
        print(f"中文繁体转换为简体...{doc[:10]}...")
    else:
        print("警告：无法识别文档语言，跳过预处理")
        return doc, 'unknown'
    
    doc = contractions.fix(doc) 
    print(f"After contractions: {doc[:100]}\n{doc[len(doc)-100:]}")
    doc_ori_count = doc_ori.count("'")
    doc_count = doc.count("'")
    print(f"Check contractions: {doc_ori_count}, {doc_count}")
    print(test_english_contractions(doc_ori, doc))                                                                 

    doc = doc.lower()
    print(f"After lower: {doc[:100]}\n{doc[len(doc)-100:]}")
    
    doc = lemmatize_text(doc)
    print(f"After lemmatization: {doc[:100]}\n{doc[len(doc)-100:]}")
    
    doc = remove_special_characters(doc, lang, remove_digits=True)
    print(f"After remove_special_characters: {doc[:100]}\n{doc[len(doc)-100:]}")    

    # remove stopwords
    doc = remove_stopwords(doc, is_lower_case=False)
    print(f"After remove_stopwords: {doc[:100]}\n{doc[len(doc)-100:]}")    


In [49]:
for doc in docs:
    lang = detect_language(doc)
    test_all(doc, lang)

File1: [Illustration:

                             GEORGE ALLEN
                               PUBLISHER

...
  CHISWICK PRESS:--CHARLES WHITTINGHAM AND CO.
                  TOOKS COURT, CHANCERY LANE, LONDON.
After strip_html_tags: [Illustration:
                             GEORGE ALLEN
                               PUBLISHER
  
  CHISWICK PRESS:--CHARLES WHITTINGHAM AND CO.
                  TOOKS COURT, CHANCERY LANE, LONDON.
After remove_accented_chars: [Illustration:
                             GEORGE ALLEN
                               PUBLISHER
  
  CHISWICK PRESS:--CHARLES WHITTINGHAM AND CO.
                  TOOKS COURT, CHANCERY LANE, LONDON.
After contractions: [Illustration:
                             GEORGE ALLEN
                               PUBLISHER
  
  CHISWICK PRESS:--CHARLES WHITTINGHAM AND CO.
                  TOOKS COURT, CHANCERY LANE, LONDON.
Check contractions: 0, 0


TypeError: test_english_contractions() takes 1 positional argument but 2 were given

In [ ]:
doc = docs[2]
lang = detect_language(doc)
test_all(doc, lang)

In [ ]:
processed_docs = []
langs = []

for doc in docs:        
    processed_doc, lang = normalize_doc(doc, isDebug=False)
    save_processed_text('./data/processed.txt', processed_doc)
    processed_docs.append(processed_doc)
    langs.append(lang)

# 测试模块

# 主函数

In [ ]:
def main(booklist):
    print("# 预处理结果报告\n") 
    
    for bookname, file_path in booklist:
        print(f"\n##=== 处理书籍：《{bookname}》===")
           
        # 1. 获取原始文本
        original_doc = fetch_gutenberg_text(file_path)
        if original_doc is None:
            print(f"无法获取文本，跳过处理")
            continue    
                
        print(f"成功获取文本:长度{len(original_doc)}字符")
        
        # 2. 执行预处理
        processed_doc, lang = normalize_doc(
            original_doc,
            html_stripping=True,
            contraction_expansion=True,
            accented_char_removal=True,
            text_lower_case=True,
            text_lemmatization=True,
            special_char_removal=True,
            stopword_removal=True,
            remove_digits=True,
            zh_simplification=True,
            isDebug=False   
        )

        print(f"预处理完成，语言：{lang}，处理后长度：{len(processed_doc)}字符")
           
        if lang == 'unknown' or not processed_doc:
            print("跳过检查（未知语言或空文本）")
            continue

        # 3. 结果输出存储
        save_processed_text(file_path, processed_doc)  # 调用保存函数

        # 4. 输出前10高频词及前20长的单词
        n, k = 10, 20
        top_n, longest_k = get_statistics(processed_doc, n=n, k=k, lang=lang)
        print(f"\n=== 输出文本前{n}高频词和前{k}长的单词 ===")
        print(f"前{n}高频词：{top_n}")
        print(f"前{k}长单词：{longest_k}\n")

        print(f"\n=== 检查预处理结果：《{bookname}》 ===")            
        
        # 获取原始文本和处理后文本
        errors = test_preprocessed_text(processed_doc, lang) if processed_doc else []
        if not errors:
            print("✅ 所有检查通过")
        else:
            print("❌ 错误：")
            for e in errors:
                print(f"- {e}")

if __name__ == "__main__":
    # 修改命令行参数解析：接收配置文件路径作为参数
    parser = argparse.ArgumentParser(description="处理书籍列表并生成结果（配置文件为JSON格式）")
    parser.add_argument('config_file', help="书籍列表配置文件路径（如 booklist.json）")
    args = parser.parse_args()
    
    # 动态获取配置文件路径
    config_path = Path(args.config_file)
    
    # 验证配置文件是否存在
    if not config_path.exists():
        print(f"错误：配置文件 '{config_path}' 不存在，请检查路径")
        exit(1)
    
    # 读取并解析配置文件
    try:
        with open(config_path, 'r', encoding='utf-8') as f:
            config = json.load(f)
        booklist = config.get('booklist', [])
        if not booklist:
            print("错误：配置文件中未找到有效的 'booklist' 列表")
            exit(1)
    except json.JSONDecodeError:
        print(f"错误：配置文件 '{config_path}' 格式错误（需为JSON格式）")
        exit(1)
    except Exception as e:
        print(f"错误：读取配置文件时发生异常 - {str(e)}")
        exit(1)
    
    # 执行主函数
    main(booklist)

usage: ipykernel_launcher.py [-h] config_file
ipykernel_launcher.py: error: the following arguments are required: config_file


SystemExit: 2

/Users/hqyang/anaconda3/envs/py39/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
%run -d Assign1-1-ans.py booklist_config.json  # -d 表示开启调试模式